## 導入
### 【課題】SVMとは何か
- どのようなときに使うのか
>主に２クラス分類のときに使うが、回帰にも使えるように拡張されている。

- どのような仕組みなのか
>プロットした特徴量を点とすると、どの点にも被らないように、できるだけ幅の広い帯を引こうとする。この帯のことをマージンと呼ぶ。このマージンを最大化し、その真ん中が判別のための決定境界になる。この境界線により２クラス分類を行う。

- どういった利点欠点があるのか
>1. 利点：①特徴量が増えても精度が高い。②外れ値の存在が境界線に影響を与えない＝モデル全体に影響を与える懸念がない。③単純に直線では分けられない非線形な問題にも対処可能である。
>2. 欠点：①多クラス分類の１対１方式においては、n数のクラスから２個を選ぶ組合せnC2分だけ計算を行うため、特徴量が増えるごとに時間がかかる。②１対他方式においては、各SVMにおいて2つのクラスのバランスが悪い。③各SVMによる予測に差が出てそれぞれの優劣を決められない。

### ※必要なライブラリをimport

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

## 【課題】データを取得する

In [2]:
df=pd.read_csv('mushrooms.csv')
df.head(10)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


## 前処理
### 【課題】データの変換
- このデータセットではターゲットも特徴量も全て文字列なので、SVMで利用できるようにそれらを整数に変換しましょう。

In [3]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [4]:
labelencoder=LabelEncoder()
for col in df.columns:
    df[col]=labelencoder.fit_transform(df[col])
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


### 【課題】データセットの分割
- testデータとtrainデータに分割しましょう。

In [5]:
X=df.iloc[:,1:23]
y=df['class']

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,
                                              random_state=0)

### 【課題】標準化
- SVMを行う前にデータの標準化をしましょう。
- また、なぜSVMを行う際に標準化が必要なのか説明してください。
>特徴量間の距離をユークリッド距離として計算するモデルの場合、ここで大きく離れている特徴量があると、そのスケールの違いに引っ張られて全体が支配されてしまう。これを避けるために、正規化することにより学習に使うすべての特徴量の範囲を一定範囲に収める（同様の尺度で計算させる）必要があるため。

In [7]:
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler

standard_scaler=StandardScaler()
X_standard_train=standard_scaler.fit_transform(X_train)
X_standard_test=standard_scaler.fit_transform(X_test)

robust_scaler=RobustScaler()
X_robust_train=robust_scaler.fit_transform(X_train)
X_robust_test=robust_scaler.fit_transform(X_test)

minmax_scaler=MinMaxScaler()
X_minmax_train=minmax_scaler.fit_transform(X_train)
X_minmax_test=minmax_scaler.fit_transform(X_test)

## ハイパーパラメータの調整
### 【課題】カーネルとは何か
- SVMにおいて重要になってくるカーネルとは何でしょうか。簡潔に説明してください。また、カーネルはどう使い分けるのが良いか調べて記述してください。
>大まかに言うと、２つのサンプル間(xi,xj)の「類似性を表す関数」であると解釈できる。
>1. 非線形なサンプルは、２次元平面上においてそのままでは線形分離できない＝マージンを引けない場合がある。
>2. 打開策 → サンプルを高次元空間へ写像する。これにより線形分離できるようにする。
>3. 写像するにはドット積をϕドット積へ置き換える計算が必要になる。しかし、計算コストがかかる。
>4. そこで登場するのがカーネル関数K(xi,xj)。カーネル関数さえわかれば、「どのように写像されているかわからなくても＝ϕを明示しなくても」計算できる。
>5. よく使われるカーネル関数にガウシアンカーネル関数(Radial Basis Function)がある。
>6. 定義式にある、底expの指数となるγは、比較する２つのサンプル間のユークリッド距離に掛かる係数。そして最適化の対象となるハイパーパラメータでもある。
>7. γにマイナス符号が定義されている。これは距離が離れている＝(xi-xj)2乗が大きいと底のexpによってゼロに近づけられる＝xiとxjは似ていないと判定される。逆の場合は１に近づけられる＝似ていると判定される。

>使い分けについては、サンプルの分布が線形ならば線形カーネル関数を、非線形ならばガウシアンカーネル関数(Radial Basis Function)を用いれば概ね対応できる。

### 【課題】コストペナルティCとは何か
- ハイパーパラメータとして設定するコストペナルティCについて簡潔に説明してください。

>制約条件の厳しさ＝「どの程度誤りを許容するか」を考える必要がある。

>これを設定するのがコストペナルティCである。

>完全を求める→高バリアンス→過学習に陥る→バリアンス・バイアスのトレードオフに起因する。

### 【課題】ハイパーパラメータを調整する
- グリッドサーチを行うことでハイパーパラメータを決定してください。なお、実行にはある程度時間がかかることが予想されます。カーネルもハイパーパラメータのひとつと考えられます。

## SVMの実行
- 【課題】学習およびテスト

In [9]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,KFold

kfold=KFold(n_splits=10,shuffle=True,random_state=0)

params={'C':[0.001,0.01,0.1,1,10,100],
        'gamma':[0.001,0.01,0.1,1,10,100],
        'kernel':["rbf", "linear"],
       }

grid=GridSearchCV(SVC(),
                  params,
                  scoring='accuracy',
                  cv=kfold,
                  n_jobs=-1
                 )

grid.fit(X_minmax_train,y_train)

print(grid.best_params_)
print(grid.best_score_)
print("Accuracy_train: %.3f"%(grid.score(X_minmax_train, y_train)))
print('-'*80)

clf=grid.best_estimator_

from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.metrics import f1_score

y_pred=clf.predict(X_minmax_test)

print("Accuracy_test: %.3f"%(accuracy_score(y_test,y_pred)))
print("Precision_test: %.3f"%(precision_score(y_test,y_pred)))
print("Recall_test: %.3f"%(recall_score(y_test, y_pred)))
print("F1_test: %.3f"%(f1_score(y_test, y_pred)))

{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
1.0
Accuracy_train: 1.000
--------------------------------------------------------------------------------
Accuracy_test: 1.000
Precision_test: 1.000
Recall_test: 1.000
F1_test: 1.000
CPU times: user 3.62 s, sys: 231 ms, total: 3.85 s
Wall time: 4min 29s
